In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import time

In [2]:
class BasicBlock(nn.Module):
    expansion = 1  # For BasicBlock, expansion factor is 1
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out


In [3]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        # Modified first conv layer: 1 input channel, 64 output channels,
        # kernel_size=3, stride=1, padding=1 (instead of 7x7 with stride 2)
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        # Do not use maxpool for small images
        # self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion)
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # x = self.maxpool(x)  # Skipped for 28x28 images

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [4]:

def resnet18(num_classes=10):
    """Constructs a ResNet-18 model."""
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)

In [5]:
num_classes = 10
num_epochs = 10
batch_size = 64
learning_rate = 0.001

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True,
                                      transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False,
                                     transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size,
                         shuffle=False)

In [7]:
# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = resnet18(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train():
    model.train()
    for epoch in range(num_epochs):
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (batch_idx + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy on the test set: {100 * correct / total:.2f}%')

if __name__ == "__main__":
    start_time = time.time()
    train()
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")

    start_time = time.time()
    test()
    end_time = time.time()
    inference_time = end_time - start_time
    print(f"Inference time: {inference_time:.2f} seconds for the entire test set")
    per_sample_inference_time = inference_time / len(test_dataset)
    print(f"Inference time per sample: {per_sample_inference_time:.6f} seconds")


Epoch [1/10], Step [100/938], Loss: 0.3792
Epoch [1/10], Step [200/938], Loss: 0.4059
Epoch [1/10], Step [300/938], Loss: 0.3602
Epoch [1/10], Step [400/938], Loss: 0.2800
Epoch [1/10], Step [500/938], Loss: 0.4655
Epoch [1/10], Step [600/938], Loss: 0.4192
Epoch [1/10], Step [700/938], Loss: 0.3125
Epoch [1/10], Step [800/938], Loss: 0.2447
Epoch [1/10], Step [900/938], Loss: 0.5698
Epoch [2/10], Step [100/938], Loss: 0.2802
Epoch [2/10], Step [200/938], Loss: 0.2631
Epoch [2/10], Step [300/938], Loss: 0.3653
Epoch [2/10], Step [400/938], Loss: 0.2321
Epoch [2/10], Step [500/938], Loss: 0.1611
Epoch [2/10], Step [600/938], Loss: 0.2110
Epoch [2/10], Step [700/938], Loss: 0.2638
Epoch [2/10], Step [800/938], Loss: 0.2235
Epoch [2/10], Step [900/938], Loss: 0.3716
Epoch [3/10], Step [100/938], Loss: 0.1518
Epoch [3/10], Step [200/938], Loss: 0.2050
Epoch [3/10], Step [300/938], Loss: 0.1706
Epoch [3/10], Step [400/938], Loss: 0.1329
Epoch [3/10], Step [500/938], Loss: 0.1674
Epoch [3/10

In [8]:
    model_path = "resnet18_fashionmnist_full.pth"
    torch.save(model.state_dict(), model_path)
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    print(f"Model size: {model_size:.2f} MB")

Model size: 42.70 MB
